In [139]:
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
import pandas as pd
from colorsys import hsv_to_rgb

In [140]:
input_file = 'MEP.xlsx'
output_file = 'MEP_Out.xlsx'

In [141]:
df = pd.read_excel(input_file)
df.head()

,UID,FromRevit_GM,Plumbing,Fire,Sprinkler,Mechanical
0,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63ce0,NaN,MP-CS,FF-AA-FH-B2-01,SP-AA-GQH,AA-OHU-B2-01
1,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cea,NaN,MP-CS-KP,FF-AA-FH-01-01,SP-AA-DQU,AA-OHU-B2-02
2,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cef,PP-ARM-VP-1,MP-CR,FF-AA-FH-02-01,SP-AA-HFQR,AA-AHU-B2-01
3,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cf6,PP-W-VL-1,MP-CR-KP,FF-AA-FH-03-01,NaN,AA-AHU-B2-02
4,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cfa,PP-W-VL-1,MP-HS,NaN,NaN,AA-FCU-B2-01


In [142]:
unique_elements = df.iloc[:, 1].dropna().unique().tolist()

In [143]:
workbook = load_workbook(input_file)
worksheet = workbook.active

**Color Creation**

In [144]:
def generate_color(n):
    colors = []
    for i in range(n):
        hue = i%n
        saturation = 0.5+(i%2)*0.25
        value = 0.8
        rgb = hsv_to_rgb(hue, saturation, value)
        hex_color = ''.join(f'{int(c*255):02X}' for c in rgb)
        colors.append(hex_color)
    return colors

In [145]:
colors = generate_color(len(unique_elements))

In [146]:
fill = [PatternFill(start_color=color, end_color=color, fill_type='solid') for color in colors]

In [147]:
fill_elements = {unique_element: fill[idx%len(fill)] for idx, unique_element in enumerate(unique_elements)}

In [148]:
matched_elements = set()

**Color fill columns that matches revit identification numbers**

In [149]:
for row in range(2, worksheet.max_row+1):
    for col in range(3, worksheet.max_column+1):
        cell = worksheet.cell(row=row, column=col)
        if cell.value in fill_elements:
            matched_elements.add(cell.value)
            cell.fill = fill_elements[cell.value]

**Color fill Identification column if it has matching elements in excel sheet elements**

In [150]:
for row in range(2, worksheet.max_row+1):
    cell = worksheet.cell(row=row, column=2)
    if cell.value in matched_elements:
        cell.fill = fill_elements[cell.value]
    else:
        cell.fill = PatternFill(fill_type=None)

**Finding unmatched elements**

In [151]:
unmatched_elements = [element for element in unique_elements if element not in matched_elements]

**Adding new column for unmatched elements**

In [152]:
if unmatched_elements is not None:
    new_column = worksheet.max_column+1
    worksheet.cell(row=1, column=new_column, value="Unmatched Elements")
for idx, value in enumerate(unmatched_elements, start=2):
    worksheet.cell(row=idx, column=new_column, value=value)

**Adding unmatched elements from excel based data to new column**

In [153]:
for col in range(3, worksheet.max_column):
    not_found_ids = []
    for row in range(2, worksheet.max_row+1):
        cell = worksheet.cell(row=row, column=col)
        if cell.value not in matched_elements:
            not_found_ids.append(cell.value)
    new_col = worksheet.max_column+1
    original_header = worksheet.cell(row=1, column=col).value
    worksheet.cell(row=1, column=new_col, value=original_header)
    for idx, value in enumerate(not_found_ids, start=2):
        worksheet.cell(row=idx, column=new_col, value=value)

**For auto fixing the size of each columns**

In [154]:
for col in worksheet.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        if cell.value is not None:
            max_length = max(max_length, len(str(cell.value)))
    worksheet.column_dimensions[col_letter].width = max_length+2

In [155]:
workbook.save(output_file)